# **Step 6 - Let's go LangChain**

**Disclaimer ❗**  

This step is **not** meant to be a LangChain tutorial, nor is it expected that you become proficient with it.

The goal is to show an example of an Agent implemented using one of the popular frameworks to appreciate some of the advantages a framework can provide.  

The decision to pick LangChain was driven by its popularity and the fact that its agent impelementation is close to what you've already seen so far, so it should look quite familiar.  

**Let's start then!**  

The first cell, apart the use of LangChain's [AzureChatOpenAI](https://python.langchain.com/v0.1/docs/integrations/chat/azure_chat_openai/) client is the same as before, you can simply run it.

In [ ]:
import os
import pytz
from dotenv import load_dotenv

# Imports
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool
from langchain.pydantic_v1 import BaseModel, Field
from datetime import datetime
from services.data_provider import DataProvider
from termcolor import cprint
from langchain.agents import AgentExecutor, create_openai_functions_agent
from typing import Optional

# Environment setup
load_dotenv()

api_key=os.environ['AZURE_OPENAI_API_KEY']
endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
api_version=os.environ['AZURE_OPENAI_API_VERSION']
local_timezone = os.environ['TIMEZONE']

llm=AzureChatOpenAI(api_key=api_key, 
                    azure_endpoint=endpoint,
                    azure_deployment=deployment,
                    api_version=api_version, 
                    temperature=0.3)

# This the data access layer
db:DataProvider=DataProvider()

This is exactly the same `system_message` of the previous step, again just run the cell...

In [ ]:
# The system message didn't change...
system_message = f'''
Role: An assistant with expertise in handling questions about operative schedule and available menu of a food delivery service.
 
Instructions:
- Provide information about the available menu and the opening schedule.
- Kindly deny any request not regarding opening schedule and available menu.
- Always assume the date and time returned by the get_current_datetime tool as the reference date for all date calculations.
- If a date is not indicated use the reference date.
- If a time is not indicated assume it is 12 PM.
- Assume the following format: 'DD/MM/YYYY HH:MM' as the standard format for dates.

Steps:
  - Opening Schedule:
      1. Always use the calculated date based on reference date to check and return the opening schedule.
      2. Always indicate the desired data and time including weekday in the response.
      
  - Available menu:
      1. When the user asks for the a menu you should respond using the following template enclosed in triple quotes:

      ```
      Menu for <day-name> <requested date>
      
      Pasta
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. ..      
      
      
      Pizza
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. .. 
      
      Today's Special Pizzas
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. ..   
      
      Drinks
      
      1. <name> - <price> 
      2. <name> - <price> 
      3. ...
      4. .. 
      
      Dessert
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. .. 
      ```  
      
      2. If the request is about specific menu entries (e.g. 'do you have pasta', 'do you offer vegetarian food' or 'are there any specials?' ), together with the menu kindle answer the question.     
      3. If no specials are available for the day, do not include that section into menu.
      4. If no menu is available, kindly reply that there you can serve any food for that day..
      5. If the request relates to an item that is not available in the menu, kindly reply that it cannot be ordered.      
  
Expectation:
  - Provide a seamless experience to the user, by providing the requested information in a kind and timely manner, including all the necessary details and guiding the user to a possible follow up step.

Narrowing:  
  1. Deny all the requests referring to a date antecedent the reference date.
'''

## Define the function arguments using LangChain

LangChain uses [Pydantic](https://docs.pydantic.dev/latest/) types to define the schema of the input parameters.  
  
So we are going to define two classes `ScheduleToolInputSchema` and  `DailyMenuInputSchema` which will represent the parameters our functions expect.  
  
We use a [Field](https://docs.pydantic.dev/latest/concepts/fields/) to provide the description of each parameter and  [Optional](https://docs.python.org/3/library/typing.html) to indicate fields which are not mandatory.

If you compare them with the information in the Tools definition of the previous step, you will see many similarities.

In [ ]:
# Lanchain uses Pydantic to define the input schema for the tools, no

# Define the input schema for the schedule-tool
class ScheduleToolInputSchema(BaseModel):
    date: str = Field(description="The date for which you want to know the schedule in format 'DD/MM/YYYY HH:MM' (e.g. '31/12/2023 15:00').")
    user_id: Optional[str] = Field(description="The optional user id in form '#<4-digits-number>'.")
    
# Define the input schema for daily menu tool
class DailyMenuInputSchema(BaseModel):
    date: str = Field(description="The date for which you want to know the available menu in format 'DD/MM/YYYY HH:MM' (e.g. '29/05/2024 17:00').")

# Define LangChain Tools using the @tool decorator 

Following are the exact same code function you've seen in the previous steps.  

The only difference here is that we identify them as [Tools](https://python.langchain.com/v0.1/docs/modules/tools/) by using the [@tool](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.tool.html#langchain_core.tools.tool) decorator and we provide the description of the function as doc-string.  

The function is linked to the input definitions we have seen before via its `arg_schema` property.  

LangChain offer [several alternatives](https://python.langchain.com/v0.2/docs/how_to/custom_tools/) to define tools, this is the simplest one.

In [ ]:
# Returns the operative schedule for the provided date and the optional user id
@tool("schedule-tool", args_schema=ScheduleToolInputSchema)
def operative_schedule(date:str, user_id:str=None):
    """Useful when you need to know the operative schedule on a specific date and time."""
    weekday= datetime.strptime(date, "%d/%m/%Y %H:%M").strftime("%A")    
    
    # Check if the user is a special client
    is_special = db.is_special_client(user_id)
    # Get the operative schedule from the database
    schedule=db.get_schedule(day=weekday, is_special=is_special) 
    # Check if the requested date/timw comes within the working schedule       
    if schedule.status == "open":        
        time= datetime.strptime(date, "%d/%m/%Y %H:%M").time()
        opening = datetime.strptime(schedule.start, "%H:%M").time()
        closing = datetime.strptime(schedule.end, "%H:%M").time()        
        if time < opening or time > closing:
            schedule.status = "closed"            
        
    return schedule

# Returns the menu for the provided date
@tool("daily-menu-tool", args_schema=DailyMenuInputSchema)
def get_menu(date:str):
    """Useful when you need to know the menu for a specific date."""
    target_date= datetime.strptime(date, "%d/%m/%Y %H:%M") 
    weekday= target_date.strftime("%A")
    
    # Get the menu from the database
    return db.get_menu(day=weekday)

# Function to get the current date and time in the format dd/mm/yyyy hh:mm
@tool("current-datetime-tool")
def get_current_datetime():
    """Useful when you need to know the current date and time for date and time based operations."""        
    local_tz = pytz.timezone(local_timezone)
    return {
        "current_datetime": datetime.now(local_tz).strftime("%A, %d/%m/%Y %H:%M")
    } 

Since we need to use them later, we create an array of tools.

In [ ]:
# We create an array containing the tools that will be used by the agent
tools=[operative_schedule, get_menu, get_current_datetime]

## Create an Agent to using the LangChain AgentExecutor

In the next cell we create a function `run_agent` in charge of processing the passed `user_request`.

Let go through it:  
1. A Langchain [prompt](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/) object is created from a series of messages, the `weird` one is probaly the [MessagePlaceHoler](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/#messagesplaceholder) that, as the name says, it's a placeholder used by LangChain to add the 'invoke function' and relative response messages you have seen before.  

2. The [create_openai_function_agent](https://python.langchain.com/v0.1/docs/templates/openai-functions-agent/) creates an agent grouping the LLM, the Tools and the prompt (remember the diagram from [step 05](05_get_menu_v2.ipynb)?  

3. [AgentExecutor](https://python.langchain.com/v0.1/docs/modules/agents/concepts/#agentexecutor) is the runtime in charge of calling the tools, adding the returned data into message history, call the llm again, etc.  

4. Invokes the agent passing the user request. (note that prompt definition uses a `{input}` placeholder)


If all this look quite complicated, **no worries** (and welcome to LangChain world 🙂), just remember that this does exactly what you did before under the hood.

In [ ]:

# Creates the agent, the framework to run it and runs it using provided user request
def run_agent(user_request:str):
    # Create a prompt object made of system message, user input and a placeholder for the agent to store information returned by the tools
    prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_message),        
        ("user", "{input}"), # {input} is a placeholder for the user input passed via invoke method of AgentExecutor
        MessagesPlaceholder(variable_name="agent_scratchpad"), 
    ])

    # Create the agent
    agent = create_openai_functions_agent(llm, tools, prompt)
    # Create the runtime environment for the agent
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # verbose=True will print the messages exchanged between the agent and the tools
    # Execute the agent
    response= agent_executor.invoke({"input": user_request})
    return response["output"]

## Invoke the Agent and observe the results 

Here you can finally see the agent in action!  

Play with `user_message` and see the output, thanks to the `verbose=True` setting of [AgentExecutor](https://python.langchain.com/v0.1/docs/modules/agents/concepts/#agentexecutor) you can see what the agent is doing in the background.

In [ ]:

# Here is the input message from the user, as usual play with it to see how the agent responds
user_message = "what can i order next Tuesday?" 


cprint(f"Input: {user_message}", "blue") 
# Run the agent with the user message
response = run_agent(user_message)
cprint (response, "yellow")

Congratulations, you now have a LangChain agent! 😎

**Note:**  
  
Doesn't matter if you did not get all the LangChain stuff, what matters is to remember that in real-world project, most of the time, all the orchestration between the LLM and its tools is done by a framework, each one in a different way, but with the same goal.

Time to continue to next step: [Notebook 07](07_order_tool.ipynb) where we'll add the tool in charge of handling orders.